### Installing libraries

In [1]:
!pip install geopy
!pip install geocoder
!pip install folium

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Scraping Neighborhood data

In [2]:
address = 'Los Angeles, United States'
url = "https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
neighborhoodList = []

# -- append the data into the list

links = []
for row in soup.find_all("div", class_="div-col"):
    for item in row.select("a"):
        if item.has_attr('title'):
            neighborhoodList.append(item.text)

la_df = pd.DataFrame({"Neighborhood": neighborhoodList})
la_df.head(10)

,Neighborhood
0,Adams-Normandie
1,Alsace
2,Angelino Heights
3,Angeles Mesa
4,Angelus Vista
5,Arleta
6,Arlington Heights
7,Arts District
8,Atwater Village
9,Baldwin Hills


In [3]:
la_df.shape

(219, 1)

In [4]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles, United States'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
coordinates = [ get_latlng(neighborhood) for neighborhood in la_df["Neighborhood"].tolist()]

In [6]:
coordinates

[[33.901238000000035, -118.29908299999994],
 [33.97902000000005, -118.41417999999999],
 [34.07029000000006, -118.25479999999999],
 [18.05302000000006, -65.87515999999994],
 [34.084375707919705, -118.26919066978292],
 [34.24905000000007, -118.43348999999995],
 [34.03989000000007, -118.32515999999998],
 [34.04193300000003, -118.23644599999994],
 [34.11970000000008, -118.25886999999994],
 [34.021570000000054, -118.36764999999997],
 [34.011716000000035, -118.34072499999996],
 [34.07044548680411, -118.20070975126488],
 [34.07044548680411, -118.20070975126488],
 [34.10915025473649, -118.32045034087673],
 [34.08361000000008, -118.43482999999998],
 [34.12306719490349, -118.43504285082464],
 [34.09653268819107, -118.40329514585207],
 [34.11793000000006, -118.44608999999997],
 [34.09544836979449, -118.42608686997163],
 [34.075496000000044, -118.35588099999995],
 [34.063720000000046, -118.26459999999997],
 [34.05138000000005, -118.40132999999997],
 [34.04004000000003, -118.21049999999997],
 [34.0

In [7]:
df_coords = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
# Merge the coordinates into the original dataframe
la_df['Latitude'] = df_coords['Latitude']
la_df['Longitude'] = df_coords['Longitude']
print(la_df.shape)
la_df

(219, 3)


,Neighborhood,Latitude,Longitude
0,Adams-Normandie,33.901238,-118.299083
1,Alsace,33.979020,-118.414180
2,Angelino Heights,34.070290,-118.254800
3,Angeles Mesa,18.053020,-65.875160
4,Angelus Vista,34.084376,-118.269191
5,Arleta,34.249050,-118.433490
6,Arlington Heights,34.039890,-118.325160
7,Arts District,34.041933,-118.236446
8,Atwater Village,34.119700,-118.258870
9,Baldwin Hills,34.021570,-118.367650


In [8]:
la_df.dtypes

Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [9]:
la_df.to_csv("la_df.csv", index=False)

In [10]:
address = 'Los Angeles'

geolocator = Nominatim(user_agent = "ExploreLA")
LA_location = geolocator.geocode(address)
LA_latitude = LA_location.latitude
LA_longitude = LA_location.longitude

print('The geograpical coordinates of Los Angeles are {}, {}.'.format(LA_latitude, LA_longitude))

The geograpical coordinates of Los Angeles are 34.0536909, -118.242766.


### Using Folium to plot neighborhoods on the map

In [11]:
mapLA = folium.Map(
    location=[LA_latitude, LA_longitude], 
    tiles='openstreetmap', 
    zoom_start=9.5, 
)

# add markers to map
for lat, lng, neighborhood in zip(la_df['Latitude'], la_df['Longitude'], la_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.Marker(
        [lat, lng],
        popup = label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7
    ).add_to(mapLA)

In [13]:
mapLA

In [14]:
mapLA.save('mapLA.html')

### Using Foursquare API

In [26]:
CLIENT_ID = 'WEN25BJ1WKZKS0LTYGLOFOSQF5U1NH2MPFBX3Q33FTD23Z0Y' 
CLIENT_SECRET = 'ECSJJ0XFAZPFEEBZCSLBPG5EUCRL3CKNKZWYELTEOFBPJZAW' 
VERSION = '20180605' 


radius=2000
LIMIT=200

In [27]:
def getVeneus(neighborhood, latitude, longitude, category=None, radius=1000):
    venues_list = []
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "v": VERSION,
        "ll": "{},{}".format(latitude, longitude),
        "radius": radius,
        "limit": LIMIT,
    }
    url = 'https://api.foursquare.com/v2/venues/search'    
    results = requests.get(url, params=params).json()

    if not results["response"]:
        return []

    for v in results["response"]['venues']:
        if not v['categories']:
            continue
        venues_list.append([
            neighborhood,
            latitude, 
            longitude, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],
            v['categories'][0]["name"]
        ])
    return venues_list

In [28]:
def getNearbyVenues(neighborhoods, latitudes, longitudes, category=None, radius=1000):
    
    venues_list=[]
    for neig, lat, lng in zip(neighborhoods, latitudes, longitudes):
        results = getVeneus(neig, lat, lng, category=category, radius=radius)
        venues_list += results
        
    if not venues_list:
        print("venue list is empty")
        return []
    
    venues_data = pd.DataFrame(venue for venue in venues_list)
    venues_data.columns = [
        'Neighborhood', 
        'Neighborhood Latitude', 
        'Neighborhood Longitude', 
        'Venue', 
        'Venue Latitude', 
        'Venue Longitude',
        'Venue Category',
    ]
    
    return venues_data

In [29]:
venues_df = getNearbyVenues(
    neighborhoods=la_df['Neighborhood'],
    latitudes=la_df['Latitude'],
    longitudes=la_df['Longitude'],
)

venue list is empty


In [30]:
venues_df.drop_duplicates(keep="first", inplace=True)
venues_df

AttributeError: 'list' object has no attribute 'drop_duplicates'

In [24]:
venues_df.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adams-Normandie,45,45,45,45,45,45


In [23]:
def collect_categories(node, categories):
    categories.append(node["name"])
    if not node["categories"]:
        return
    for sub_node in node['categories']:
        collect_categories(sub_node, categories)
categories_list = {}
for i in get_categories():
    categories = []
    collect_categories(i, categories)
    categories_list[i["name"]] = categories

KeyError: 'categories'

In [3]:
venueCat = []

for venue_category in venues_df["Venue Category"]:
    for key in categories_list.keys():
        if venue_category in categories_list[key]:
            venueCat.append(key)

venues_df["General Venue Category"] = venueCat

venues_df.head(20)

NameError: name 'venues_df' is not defined

In [4]:
colorDict = {
    'Outdoors & Recreation': 'cadetblue',
    'Food': 'darkgreen',
    'Nightlife Spot': 'beige',
}

In [5]:
from folium.plugins import MarkerCluster

venueMap = folium.Map(
    location=[LA_latitude, LA_longitude], 
    tiles='Stamen Toner', 
    zoom_start=10
)

markCluster = MarkerCluster().add_to(venueMap)

for lat, lng, cat in zip(venues_df['Venue Latitude'],
                         venues_df['Venue Longitude'],
                         venues_df['General Venue Category']):  
    if cat in colorDict:
        folium.Marker(
            location=[lat, lng],
            icon=folium.Icon(color=colorDict[cat]),
        ).add_to(markCluster)

NameError: name 'folium' is not defined